<a href="https://colab.research.google.com/github/OttmarIvey/MNIST-con-TensorFLow/blob/main/MNIST_con_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

### Cargamos el dataset de MNIST

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist',with_info=True,as_supervised=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
mnist_entrenamiento = mnist_dataset['train']
mnist_prueba =  mnist_dataset['test']

num_muestras_validacion = 0.1*mnist_info.splits['train'].num_examples #Extrae el 10% del dataset de prueba
num_muestras_validacion = tf.cast(num_muestras_validacion,tf.int64) #Hacemos que el numero se4a un enetero

num_muestras_prueba = mnist_info.splist('test').num_examples #Guardamos el numero de muestras de prueba en una variable
num_muestras_prueba = tf.cast(num_muestras_prueba,tf.int64) #Nos aseguramos de que sea un numero entero

#Ahora creamos una funcion que escale las entradas entre 0 y 1

def escala(image, label):
  image = tf.cast(image,tf.float32) #Para que todos los numeros sean reales
  image /=255. #Dividimos cada elemento de la imagen entre 255 ya que es el maximo valor que puede tener.
  return(image,label)

datos_escalados_entrenamiento_validacion = mnist_entrenamiento.map(escala)
datos_prueba = mnist_prueba.map(escala)

#Ahora vamos a permutar los datos del dataset MNIST aleatoriamente ya que es posible que este organizado de alguna forma.
#lo cual confundira al modelo.

BUFFER_SIZE = 10000


